In [1]:
#Import Sioux City Data
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_f7602a2efe1946178355330fdd8d74a0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Z-iL5M8gm6Hyxpmlep-G7T_Wr7Gqs-uwPpYVorNAd6BV',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_f7602a2efe1946178355330fdd8d74a0.get_object(Bucket='myfirstproject-donotdelete-pr-jqve05ji1rjj9i',Key='Sioux_City_Zip.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,NAME,ZIPCODE,LAT,LON,INCOME
0,Downtown Sioux City,51101,42.49156,-96.39943,"17,649"
1,North Sioux City,51105,42.51332,-96.37071,"33,390"
2,Briar Cliffs,51104,42.53499,-96.40241,"59,096"
3,Riverside,51103,42.52066,-96.44160,"38,181"
4,Singing Hills,51106,42.46039,-96.32263,"54,415"


In [2]:
#Import Sioux Falls Data
body = client_f7602a2efe1946178355330fdd8d74a0.get_object(Bucket='myfirstproject-donotdelete-pr-jqve05ji1rjj9i',Key='Sioux_Falls_Zip-1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()



,NAME,ZIP,LAT,LON,INCOME
0,Downtown Sioux Falls,57103,43.54379,-96.69381,"56,319"
1,Central Sioux Falls,57105,43.52211,-96.73395,"57,295"
2,North Sioux Falls,57104,43.59735,-96.70126,"36,850"
3,Northwest Sioux Falls,57107,43.60164,-96.82520,"61,213"
4,Southwest Sioux Falls,57106,43.50241,-96.83039,"61,620"


In [3]:
#Shape of Sioux City
df_data_1.shape


(8, 5)

In [4]:
#Shape of Sioux Falls
df_data_2.shape

(8, 5)

In [5]:
#Importing Necessary Libraries 
from geopy.geocoders import Nominatim
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries have been imported.')

Libraries have been imported.


In [7]:
# My Four Square Credientials
CLIENT_ID = 'CZLUC2QLCUAWUCC2FOOXJ25KVRKQINBGHEQ2ZRIBSEJ1ZCLY' # your Foursquare ID
CLIENT_SECRET = 'TLQBSB32F5LKTG4CA1GBPOA4KQGKPEWBXTLGA5YA3MXXBK3Y' # your Foursquare Secret
VERSION = '20181224' # Foursquare API version

print('My Four Quare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My Four Quare credentails:
CLIENT_ID: CZLUC2QLCUAWUCC2FOOXJ25KVRKQINBGHEQ2ZRIBSEJ1ZCLY
CLIENT_SECRET:TLQBSB32F5LKTG4CA1GBPOA4KQGKPEWBXTLGA5YA3MXXBK3Y


In [8]:
#This is how I am going to get data from API Foursquare 
def getNearbyVenues(names, latitudes, longitudes, radius=7500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
#Getting the actual data
sux_venues=getNearbyVenues(df_data_1['NAME'], df_data_1['LAT'], df_data_1['LON '])
fsd_venues=getNearbyVenues(df_data_2['NAME'], df_data_2['LAT'], df_data_2['LON'])

In [10]:
#Raw Data Sioux City
sux_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Sioux City,42.49156,-96.39943,Rebos,42.494287,-96.396224,Mexican Restaurant
1,Downtown Sioux City,42.49156,-96.39943,Jitters,42.493512,-96.397613,Coffee Shop
2,Downtown Sioux City,42.49156,-96.39943,Buffalo Alice,42.494148,-96.396742,Bar
3,Downtown Sioux City,42.49156,-96.39943,Norm Waitt Sr. YMCA,42.485697,-96.404671,Gym / Fitness Center
4,Downtown Sioux City,42.49156,-96.39943,Orpheum Theater,42.496182,-96.404819,Concert Hall


In [12]:
#Raw Data Sioux Falls
fsd_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Sioux Falls,43.54379,-96.69381,Qdoba Mexican Grill,43.546909,-96.696105,Mexican Restaurant
1,Downtown Sioux Falls,43.54379,-96.69381,Giliberto's Mexican Taco Shop East,43.546210,-96.682144,Mexican Restaurant
2,Downtown Sioux Falls,43.54379,-96.69381,Pita Pit,43.546276,-96.696164,Sandwich Place
3,Downtown Sioux Falls,43.54379,-96.69381,Bagel Boy,43.529699,-96.685699,Bagel Shop
4,Downtown Sioux Falls,43.54379,-96.69381,Hy-Vee,43.547882,-96.687972,Grocery Store


In [91]:
sux_venues.shape

(721, 7)

In [92]:
fsd_venues.shape

(650, 7)

In [93]:
#Counting number of venues in Sioux City 
sux_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Briar Cliffs,100,100,100,100,100,100
Dakota Dunes SD,47,47,47,47,47,47
Downtown Sioux City,100,100,100,100,100,100
North Sioux City,100,100,100,100,100,100
Riverside,100,100,100,100,100,100
Sergant Bluff,94,94,94,94,94,94
Singing Hills,80,80,80,80,80,80
South Sioux Nebraska,100,100,100,100,100,100


In [94]:
#Counting Number of venues in Sioux Falls 
fsd_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Sioux Falls,100,100,100,100,100,100
Downtown Sioux Falls,100,100,100,100,100,100
East Sioux Falls,95,95,95,95,95,95
North Sioux Falls,100,100,100,100,100,100
Northwest Sioux Falls,38,38,38,38,38,38
South Sioux falls,100,100,100,100,100,100
Southwest Sioux Falls,100,100,100,100,100,100
Tea,17,17,17,17,17,17


In [13]:
# Maching Learnigng One hot 
sux_onehot = pd.get_dummies(sux_venues[['Venue Category']], prefix="", prefix_sep="")
sux_onehot['Neighborhood'] = sux_venues['Neighborhood'] 
fixed_columns = [sux_onehot.columns[-1]] + list(sux_onehot.columns[:-1])
sux_onehot = sux_onehot[fixed_columns]

sux_onehot.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Baseball Field,Big Box Store,Bookstore,Brewery,Buffet,Burger Joint,Burrito Place,Candy Store,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Football Field,Concert Hall,Convenience Store,Cosmetics Shop,Department Store,Diner,Discount Store,Dive Bar,Dog Run,Electronics Store,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hockey Arena,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Lake,Lingerie Store,Liquor Store,Mexican Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,New American Restaurant,Optical Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool Hall,Rest Area,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Sporting Goods Shop,Stadium,State / Provincial Park,Steakhouse,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store
0,Downtown Sioux City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Downtown Sioux City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Downtown Sioux City,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Downtown Sioux City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Downtown Sioux City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
sux_grouped = sux_onehot.groupby('Neighborhood').mean().reset_index()
sux_grouped.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Baseball Field,Big Box Store,Bookstore,Brewery,Buffet,Burger Joint,Burrito Place,Candy Store,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Football Field,Concert Hall,Convenience Store,Cosmetics Shop,Department Store,Diner,Discount Store,Dive Bar,Dog Run,Electronics Store,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hockey Arena,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Lake,Lingerie Store,Liquor Store,Mexican Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,New American Restaurant,Optical Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool Hall,Rest Area,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Sporting Goods Shop,Stadium,State / Provincial Park,Steakhouse,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store
0,Briar Cliffs,0.01,0.030000,0.00,0.00,0.010000,0.01,0.01,0.100000,0.0,0.0,0.00,0.01,0.01,0.040000,0.010000,0.01,0.010000,0.00,0.0,0.01,0.080000,0.0,0.01,0.00,0.010000,0.0,0.0,0.03,0.01,0.01,0.00,0.00,0.000000,0.01,0.010000,0.01,0.00,0.0,0.040000,0.01,0.000000,0.01,0.030000,0.000000,0.02,0.03,0.01,0.01,0.030000,0.010000,0.00,0.01,0.01,0.0,0.000000,0.00,0.000000,0.070000,0.00,0.0,0.01,0.000000,0.01,0.000000,0.02,0.01,0.01,0.080000,0.0,0.01,0.0,0.010000,0.010000,0.030000,0.00,0.0,0.00,0.01,0.010000,0.01,0.01,0.00,0.01,0.00,0.0,0.010000,0.0
1,Dakota Dunes SD,0.00,0.081633,0.00,0.00,0.020408,0.00,0.00,0.122449,0.0,0.0,0.00,0.00,0.00,0.020408,0.020408,0.00,0.020408,0.00,0.0,0.00,0.061224,0.0,0.00,0.00,0.020408,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.020408,0.00,0.040816,0.00,0.00,0.0,0.081633,0.00,0.020408,0.00,0.020408,0.040816,0.00,0.00,0.00,0.00,0.061224,0.020408,0.00,0.00,0.00,0.0,0.020408,0.00,0.020408,0.040816,0.00,0.0,0.00,0.020408,0.00,0.040816,0.00,0.00,0.00,0.081633,0.0,0.00,0.0,0.020408,0.020408,0.020408,0.00,0.0,0.00,0.00,0.020408,0.00,0.00,0.00,0.00,0.00,0.0,0.020408,0.0
2,Downtown Sioux City,0.00,0.020000,0.00,0.01,0.020000,0.01,0.02,0.090000,0.0,0.0,0.01,0.01,0.00,0.030000,0.010000,0.01,0.000000,0.01,0.0,0.01,0.050000,0.0,0.01,0.01,0.010000,0.0,0.0,0.00,0.01,0.01,0.01,0.01,0.000000,0.01,0.020000,0.01,0.01,0.0,0.040000,0.01,0.000000,0.01,0.030000,0.000000,0.03,0.02,0.01,0.01,0.030000,0.010000,0.01,0.01,0.01,0.0,0.000000,0.01,0.000000,0.070000,0.01,0.0,0.01,0.000000,0.01,0.000000,0.02,0.00,0.02,0.060000,0.0,0.01,0.0,0.010000,0.010000,0.030000,0.00,0.0,0.01,0.01,0.000000,0.01,0.01,0.00,0.01,0.01,0.0,0.010000,0.0
3,North Sioux City,0.00,0.020000,0.00,0.01,0.010000,0.01,0.02,0.100000,0.0,0.0,0.00,0.01,0.01,0.030000,0.010000,0.01,0.000000,0.01,0.0,0.01,0.070000,0.0,0.01,0.01,0.020000,0.0,0.0,0.01,0.01,0.01,0.00,0.01,0.000000,0.01,0.000000,0.01,0.00,0.0,0.030000,0.01,0.000000,0.01,0.040000,0.000000,0.03,0.02,0.01,0.01,0.030000,0.010000,0.00,0.01,0.01,0.0,0.000000,0.00,0.000000,0.070000,0.01,0.0,0.01,0.000000,0.01,0.000000,0.02,0.00,0.02,0.080000,0.0,0.01,0.0,0.010000,0.010000,0.030000,0.01,0.0,0.00,0.01,0.000000,0.01,0.01,0.00,0.01,0.01,0.0,0.010000,0.0
4,Riverside,0.01,0.030000,0.01,0.00,0.010000,0.01,0.01,0.120000,0.0,0.0,0.00,0.01,0.01,0.040000,0.010000,0.01,0.010000,0.01,0.0,0.01,0.080000,0.0,0.01,0.01,0.010000,0.0,0.0,0.00,0.01,0.01,0.00,0.00,0.000000,0.01,0.010000,0.00,0.00,0.0,0.050000,0.01,0.010000,0.00,0.040000,0.010000,0.02,0.02,0.01,0.01,0.030000,0.010000,0.00,0.01,0.01,0.0,0.000000,0.00,0.000000,0.070000,0.00,0.0,0.01,0.000000,0.01,0.000000,0.01,0.00,0.01,0.060000,0.0,0.01,0.0,0.010000,0.010000,0.020000,0.00,0.0,0.00,0.01,0.010000,0.01,0.01,0.01,0.01,0.01,0.0,0.010000,0.0


In [15]:
# Define a function for top venus for a neighborhood
num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
sux_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
sux_neighborhoods_venues_sorted['Neighborhood'] = sux_grouped['Neighborhood']

for ind in np.arange(sux_grouped.shape[0]):
    sux_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sux_grouped.iloc[ind, :], num_top_venues)

sux_neighborhoods_venues_sorted.head(8)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Briar Cliffs,Bar,Pizza Place,Coffee Shop,Mexican Restaurant,Burger Joint,Gas Station,Sandwich Place,American Restaurant,Discount Store,Grocery Store
1,Dakota Dunes SD,Bar,Pizza Place,American Restaurant,Gas Station,Coffee Shop,Hotel,Optical Shop,Fast Food Restaurant,Mexican Restaurant,Gym
2,Downtown Sioux City,Bar,Mexican Restaurant,Pizza Place,Coffee Shop,Gas Station,Sandwich Place,Hotel,Gym / Fitness Center,Grocery Store,Burger Joint
3,North Sioux City,Bar,Pizza Place,Mexican Restaurant,Coffee Shop,Grocery Store,Burger Joint,Hotel,Gas Station,Sandwich Place,Gym / Fitness Center
4,Riverside,Bar,Coffee Shop,Mexican Restaurant,Pizza Place,Gas Station,Burger Joint,Grocery Store,Hotel,American Restaurant,Gym / Fitness Center
5,Sergant Bluff,Pizza Place,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Discount Store,Grocery Store,Chinese Restaurant,Gas Station,Cosmetics Shop,Bar
6,Singing Hills,Bar,Mexican Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Cosmetics Shop,Grocery Store,Burger Joint,Chinese Restaurant,Pharmacy
7,South Sioux Nebraska,Bar,Coffee Shop,Gas Station,Mexican Restaurant,Pizza Place,Hotel,Burger Joint,American Restaurant,Sandwich Place,Fast Food Restaurant


In [17]:
from sklearn.cluster import KMeans

kclusters = 5

sux_grouped_clustering = sux_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sux_grouped_clustering)

kmeans.labels_[0:10] 

sux_merged = df_data_1

sux_merged['Cluster Labels'] = kmeans.labels_

sux_merged = sux_merged.join(sux_neighborhoods_venues_sorted.set_index('Neighborhood'), on='NAME')

sux_merged.head(8)

,NAME,ZIPCODE,LAT,LON,INCOME,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Sioux City,51101,42.49156,-96.39943,"17,649",1,Bar,Mexican Restaurant,Pizza Place,Coffee Shop,Gas Station,Sandwich Place,Hotel,Gym / Fitness Center,Grocery Store,Burger Joint
1,North Sioux City,51105,42.51332,-96.37071,"33,390",0,Bar,Pizza Place,Mexican Restaurant,Coffee Shop,Grocery Store,Burger Joint,Hotel,Gas Station,Sandwich Place,Gym / Fitness Center
2,Briar Cliffs,51104,42.53499,-96.40241,"59,096",1,Bar,Pizza Place,Coffee Shop,Mexican Restaurant,Burger Joint,Gas Station,Sandwich Place,American Restaurant,Discount Store,Grocery Store
3,Riverside,51103,42.52066,-96.44160,"38,181",1,Bar,Coffee Shop,Mexican Restaurant,Pizza Place,Gas Station,Burger Joint,Grocery Store,Hotel,American Restaurant,Gym / Fitness Center
4,Singing Hills,51106,42.46039,-96.32263,"54,415",4,Bar,Mexican Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Cosmetics Shop,Grocery Store,Burger Joint,Chinese Restaurant,Pharmacy
5,Sergant Bluff,51111,42.41448,-96.38793,"56,318",3,Pizza Place,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Discount Store,Grocery Store,Chinese Restaurant,Gas Station,Cosmetics Shop,Bar
6,South Sioux Nebraska,68776,42.47802,-96.46125,"51,389",2,Bar,Coffee Shop,Gas Station,Mexican Restaurant,Pizza Place,Hotel,Burger Joint,American Restaurant,Sandwich Place,Fast Food Restaurant
7,Dakota Dunes SD,57049,42.52041,-96.50418,"73,567",4,Bar,Pizza Place,American Restaurant,Gas Station,Coffee Shop,Hotel,Optical Shop,Fast Food Restaurant,Mexican Restaurant,Gym


In [18]:
# Sioux Falls one hot encoding 
fsd_onehot = pd.get_dummies(fsd_venues[['Venue Category']], prefix="", prefix_sep="")

fsd_onehot['Neighborhood'] = fsd_venues['Neighborhood'] 

fixed_columns = [fsd_onehot.columns[-1]] + list(fsd_onehot.columns[:-1])
fsd_onehot = fsd_onehot[fixed_columns]

fsd_onehot.head()

,Neighborhood,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Big Box Store,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Business Service,Butcher,Café,Campground,Carpet Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convention Center,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Electronics Store,Eye Doctor,Fabric Shop,Farmers Market,Fast Food Restaurant,Fireworks Store,Food,Food & Drink Shop,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health Food Store,History Museum,Hobby Shop,Hockey Rink,Home Service,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Music Store,Nail Salon,New American Restaurant,Noodle House,Organic Grocery,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Record Shop,Rental Service,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Theater,Theme Restaurant,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Downtown Sioux Falls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Downtown Sioux Falls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Downtown Sioux Falls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Downtown Sioux Falls,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Downtown Sioux Falls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
fsd_grouped = fsd_onehot.groupby('Neighborhood').mean().reset_index()
fsd_grouped.head()

,Neighborhood,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Big Box Store,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Business Service,Butcher,Café,Campground,Carpet Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convention Center,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Electronics Store,Eye Doctor,Fabric Shop,Farmers Market,Fast Food Restaurant,Fireworks Store,Food,Food & Drink Shop,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health Food Store,History Museum,Hobby Shop,Hockey Rink,Home Service,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Music Store,Nail Salon,New American Restaurant,Noodle House,Organic Grocery,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Record Shop,Rental Service,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Theater,Theme Restaurant,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Central Sioux Falls,0.040000,0.010000,0.000000,0.000000,0.01,0.020000,0.020000,0.030000,0.0,0.00,0.000000,0.01,0.01,0.01,0.030000,0.02,0.010000,0.000000,0.0,0.040000,0.000000,0.010000,0.020000,0.01,0.000000,0.060000,0.000000,0.00,0.010000,0.01,0.000000,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.01,0.00,0.060000,0.000000,0.000000,0.0,0.01,0.000000,0.01,0.000000,0.000000,0.01,0.010000,0.0,0.010000,0.01,0.020000,0.010000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.00,0.020000,0.01,0.000000,0.01,0.0,0.000000,0.010000,0.000000,0.010000,0.01,0.00,0.000000,0.050000,0.010000,0.000000,0.000000,0.010000,0.00,0.000000,0.0,0.03,0.000000,0.01,0.000000,0.020000,0.01,0.000000,0.000000,0.030000,0.000000,0.000000,0.020000,0.000000,0.040000,0.010000,0.01,0.0,0.000000,0.000000,0.000000,0.0,0.01,0.01,0.0,0.010000,0.000000,0.000000,0.0,0.000000,0.000000,0.01,0.01,0.0,0.01,0.000000,0.000000,0.000000,0.01,0.0,0.01,0.0,0.000000,0.0,0.01
1,Downtown Sioux Falls,0.050000,0.010000,0.010000,0.000000,0.01,0.020000,0.030000,0.060000,0.0,0.01,0.000000,0.00,0.00,0.00,0.020000,0.00,0.010000,0.000000,0.0,0.020000,0.000000,0.010000,0.020000,0.01,0.010000,0.020000,0.000000,0.01,0.000000,0.00,0.000000,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.01,0.030000,0.000000,0.000000,0.0,0.01,0.000000,0.00,0.000000,0.010000,0.01,0.000000,0.0,0.040000,0.00,0.010000,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.020000,0.01,0.010000,0.00,0.000000,0.01,0.0,0.010000,0.020000,0.000000,0.010000,0.00,0.01,0.000000,0.060000,0.010000,0.000000,0.000000,0.010000,0.01,0.000000,0.0,0.03,0.010000,0.01,0.000000,0.030000,0.01,0.000000,0.010000,0.030000,0.010000,0.000000,0.010000,0.000000,0.050000,0.010000,0.01,0.0,0.000000,0.010000,0.000000,0.0,0.01,0.00,0.0,0.010000,0.000000,0.000000,0.0,0.010000,0.000000,0.02,0.01,0.0,0.00,0.000000,0.000000,0.020000,0.01,0.0,0.01,0.0,0.000000,0.0,0.00
2,East Sioux Falls,0.030928,0.020619,0.010309,0.000000,0.00,0.010309,0.020619,0.051546,0.0,0.00,0.010309,0.00,0.00,0.00,0.010309,0.00,0.010309,0.010309,0.0,0.000000,0.030928,0.010309,0.020619,0.00,0.010309,0.030928,0.010309,0.00,0.010309,0.00,0.010309,0.020619,0.00,0.00

In [20]:

num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

fsd_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
fsd_neighborhoods_venues_sorted['Neighborhood'] = fsd_grouped['Neighborhood']

for ind in np.arange(fsd_grouped.shape[0]):
    fsd_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(fsd_grouped.iloc[ind, :], num_top_venues)

fsd_neighborhoods_venues_sorted.head(8)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Sioux Falls,Coffee Shop,Fast Food Restaurant,Mexican Restaurant,Café,Sandwich Place,American Restaurant,Brewery,Bar,New American Restaurant,Pizza Place
1,Downtown Sioux Falls,Bar,Mexican Restaurant,American Restaurant,Sandwich Place,Grocery Store,Bakery,Fast Food Restaurant,New American Restaurant,Park,Pizza Place
2,East Sioux Falls,Park,Bar,Pizza Place,Sandwich Place,Mexican Restaurant,Grocery Store,American Restaurant,Coffee Shop,Campground,Department Store
3,North Sioux Falls,Mexican Restaurant,Sandwich Place,American Restaurant,Fast Food Restaurant,Bakery,Bar,Brewery,New American Restaurant,Park,Hotel
4,Northwest Sioux Falls,Hotel,Sandwich Place,Athletics & Sports,Bar,Café,Discount Store,Fast Food Restaurant,General Entertainment,American Restaurant,Home Service
5,South Sioux falls,Park,Fast Food Restaurant,Sandwich Place,Coffee Shop,Grocery Store,Gym,Bar,Pizza Place,Café,Chinese Restaurant
6,Southwest Sioux Falls,Coffee Shop,Fast Food Restaurant,Burger Joint,American Restaurant,Pizza Place,Mexican Restaurant,Liquor Store,Ice Cream Shop,Sandwich Place,Furniture / Home Store
7,Tea,Gas Station,Water Park,Discount Store,Lounge,Park,Drugstore,Steakhouse,Construction & Landscaping,Baseball Field,Pizza Place


In [22]:
#K-Clustering 
from sklearn.cluster import KMeans

kclusters = 5

fsd_grouped_clustering = fsd_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(fsd_grouped_clustering)

kmeans.labels_[0:10] 

fsd_merged = df_data_2

fsd_merged['Cluster Labels'] = kmeans.labels_

fsd_merged = fsd_merged.join(fsd_neighborhoods_venues_sorted.set_index('Neighborhood'), on='NAME')

fsd_merged.head(8)

,NAME,ZIP,LAT,LON,INCOME,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Sioux Falls,57103,43.54379,-96.69381,"56,319",2,Bar,Mexican Restaurant,American Restaurant,Sandwich Place,Grocery Store,Bakery,Fast Food Restaurant,New American Restaurant,Park,Pizza Place
1,Central Sioux Falls,57105,43.52211,-96.73395,"57,295",0,Coffee Shop,Fast Food Restaurant,Mexican Restaurant,Café,Sandwich Place,American Restaurant,Brewery,Bar,New American Restaurant,Pizza Place
2,North Sioux Falls,57104,43.59735,-96.70126,"36,850",0,Mexican Restaurant,Sandwich Place,American Restaurant,Fast Food Restaurant,Bakery,Bar,Brewery,New American Restaurant,Park,Hotel
3,Northwest Sioux Falls,57107,43.60164,-96.82520,"61,213",0,Hotel,Sandwich Place,Athletics & Sports,Bar,Café,Discount Store,Fast Food Restaurant,General Entertainment,American Restaurant,Home Service
4,Southwest Sioux Falls,57106,43.50241,-96.83039,"61,620",4,Coffee Shop,Fast Food Restaurant,Burger Joint,American Restaurant,Pizza Place,Mexican Restaurant,Liquor Store,Ice Cream Shop,Sandwich Place,Furniture / Home Store
5,South Sioux falls,57108,43.47331,-96.68963,"94,094",3,Park,Fast Food Restaurant,Sandwich Place,Coffee Shop,Grocery Store,Gym,Bar,Pizza Place,Café,Chinese Restaurant
6,East Sioux Falls,57110,43.54969,-96.63113,"68,409",2,Park,Bar,Pizza Place,Sandwich Place,Mexican Restaurant,Grocery Store,American Restaurant,Coffee Shop,Campground,Department Store
7,Tea,57064,43.46329,-96.87032,"81,845",1,Gas Station,Water Park,Discount Store,Lounge,Park,Drugstore,Steakhouse,Construction & Landscaping,Baseball Field,Pizza Place


In [116]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  38.01 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  27.12 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  28.08 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  35.10 MB/s
